In [1]:
pip install rfdetr


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
from pathlib import Path

import cv2
import numpy as np
import torch
from PIL import Image
from rfdetr import RFDETRBase

from boxmot import BotSort

# Load RFDETR model
model = RFDETRBase(device='cpu')

# Initialize the tracker
tracker = BotSort(
    reid_weights=Path("osnet_x0_25_msmt17.pt"),  # Path to ReID model
    device=torch.device("cpu"),  # Change to 'cuda' if using GPU
    half=False
)

# Open the video file (0 for webcam or path to a video file)
vid = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = vid.read()
    if not ret:
        break

    # Convert frame to PIL Image format for RFDETR
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image = Image.fromarray(frame_rgb)

    # Perform detection
    detections = model.predict(image, threshold=0.5)
    
    # Convert detections to numpy array in format: (x1, y1, x2, y2, conf, cls)
    dets = np.column_stack((
        detections.xyxy,  # Bounding boxes (x1, y1, x2, y2)
        detections.confidence,  # Confidence scores
        detections.class_id.astype(int)  # Class IDs
    ))

    # Update the tracker
    res = tracker.update(dets, frame)  # M X (x1, y1, x2, y2, id, conf, cls, ind)

    # Plot tracking results on the image
    tracker.plot_results(frame, show_trajectories=True)

    cv2.imshow('BoXMOT + Torchvision', frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
vid.release()
cv2.destroyAllWindows()

2025-03-28 12:02:18.458 | INFO     | boxmot.utils.torch_utils:select_device:52 - Yolo Tracking v12.0.2 🚀 Python-3.11.5 torch-2.2.2CPU
2025-03-28 12:02:18.479 | SUCCESS  | boxmot.appearance.reid_model_factory:load_pretrained_weights:183 - Loaded pretrained weights from osnet_x0_25_msmt17.pt


Loading pretrain weights
